# Homework2

Team: AiCoders
Members:
* Alessio_Parato
* Jan31415
* *Vel*


Task: The goal is to design and implement forecasting models to learn how to exploit past observations in the input sequences to correctly predict the future.


## Connection and Imports

### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/Homework2/'
%cd /content/drive/MyDrive/Colab Notebooks/Homework2

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/Homework2


### Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.model_selection import train_test_split

2.15.0


In [ ]:
def save_model(model, model_name, path, mse=None, val_mse=None, mae=None, val_mae=None, plot=False):
  #
  #the functions saves the model and the history of a fit
  #
  if plot:
    tfk.utils.plot_model(model, to_file=path+'/model_pic.png', show_shapes=True, expand_nested=True)
  model.save(path+'/'+model_name)
  pd.DataFrame(mse).to_csv(path+'/'+model_name+'/mse.csv')
  pd.DataFrame(mae).to_csv(path+'/'+model_name+'/mae.csv')
  pd.DataFrame(val_mse).to_csv(path+'/'+model_name+'/val_mse.csv')
  pd.DataFrame(val_mae).to_csv(path+'/'+model_name+'/val_mae.csv')

def load_model(path):
  #
  #the functions loads the model and the history of a fit
  #
  model = tfk.models.load_model(path)
  mse = pd.read_csv(path+'/mse.csv')
  val_mse = pd.read_csv(path+'/val_mse.csv')
  mae = pd.read_csv(path+'/mae.csv')
  val_mae = pd.read_csv(path+'/val_mae.csv')

  return (model, mse, val_mse, mae, val_mae)

## Load data

* 'training_data.npy': numpy array of shape (48000, 2776). 48000 time series of length 2776.
* 'valid_periods.npy': numpy array of type (48000, 2), containing for each of the time series the start and end index of the current series, i.e. the part without padding.
* 'categories.npy': numpy array of shape (48000,), containing for each of the time series the code of its category. The possible categories are in {'A', 'B', 'C', 'D', 'E', 'F'}.

In [ ]:
#loads the unzipped dataset and divides it into data and labels
t_d = np.load('Data/training_data.npy')
cate = np.load('Data/categories.npy')
v_p = np.load('Data/valid_periods.npy')
len_v_p = v_p[:,1]-v_p[:,0]
# Print shapes of the datasets
print(f"training shape: {t_d.shape}")
print(f"categories shape: {cate.shape}")
print(f"valid_periods shape: {v_p.shape}")

training shape: (48000, 2776)
categories shape: (48000,)
valid_periods shape: (48000, 2)


### Process the dataset and Split in train, validation and test



In [ ]:
A = np.reshape((np.where(cate=='A')), 5728)
A_train, A_v_p, A_len_v_p = t_d[A], v_p[A], len_v_p[A]
B = np.reshape((np.where(cate=='B')), 10987)
B_train, B_v_p, B_len_v_p = t_d[B], v_p[B], len_v_p[B]
C = np.reshape((np.where(cate=='C')), 10017)
C_train, C_v_p, C_len_v_p = t_d[C], v_p[C], len_v_p[C]
D = np.reshape((np.where(cate=='D')), 10016)
D_train, D_v_p, D_len_v_p = t_d[D], v_p[D], len_v_p[D]
E = np.reshape((np.where(cate=='E')), 10975)
E_train, E_v_p, E_len_v_p = t_d[E], v_p[E], len_v_p[E]
F = np.reshape((np.where(cate=='F')), 277)
F_train, F_v_p, F_len_v_p = t_d[F], v_p[F], len_v_p[F]

In [ ]:
def build_sequences(data, valid_periods, window=200, telescope=9, step=209):
    dataset = []
    labels = []
    da = data.copy()
    va_p = valid_periods.copy()

    for i in range(da.shape[0]):
      series = da[i,va_p[i,0]:va_p[i,1]]
      idx = 0
      while(idx+window+telescope < series.shape[0]):
        dataset.append(series[idx:idx+window])
        labels.append(series[idx+window:idx+window+telescope])
        idx = idx+step

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels
def build_sequences2(data, window=200, telescope=9, step=209):
    dataset = []
    labels = []
    da = data.copy()

    for series in da:
      idx = 0
      while(idx+window+telescope < series.shape[0]):
        dataset.append(series[idx:idx+window])
        labels.append(series[idx+window:idx+window+telescope])
        idx = idx+step

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
A_x, A_y = build_sequences(t_d[A], v_p[A], telescope=18, step=77)
print(f'Number of Training: {A_x.shape[0]}')
B_x, B_y = build_sequences(t_d[B], v_p[B],telescope=18, step=77)
print(f'Number of Training: {B_x.shape[0]}')
C_x, C_y = build_sequences(t_d[C], v_p[C],telescope=18, step=77)
print(f'Number of Training: {C_x.shape[0]}')
D_x, D_y = build_sequences(t_d[D], v_p[D],telescope=18, step=77)
print(f'Number of Training: {D_x.shape[0]}')
E_x, E_y = build_sequences(t_d[E], v_p[E],telescope=18, step=77)
print(f'Number of Training: {E_x.shape[0]}')
F_x, F_y = build_sequences(t_d[F], v_p[F],telescope=18, step=77)
print(f'Number of Training: {F_x.shape[0]}')

Number of Training: 6765
Number of Training: 5018
Number of Training: 7630
Number of Training: 9801
Number of Training: 6296
Number of Training: 196


In [ ]:
test_size = 0.2
Ax_t_v, Ax_t, Ay_t_v, Ay_t = train_test_split(A_x, A_y, random_state=seed, test_size=test_size)
Bx_t_v, Bx_t, By_t_v, By_t = train_test_split(B_x, B_y, random_state=seed, test_size=test_size)
Cx_t_v, Cx_t, Cy_t_v, Cy_t = train_test_split(C_x, C_y, random_state=seed, test_size=test_size)
Dx_t_v, Dx_t, Dy_t_v, Dy_t = train_test_split(D_x, D_y, random_state=seed, test_size=test_size)
Ex_t_v, Ex_t, Ey_t_v, Ey_t = train_test_split(E_x, E_y, random_state=seed, test_size=test_size)
Fx_t_v, Fx_t, Fy_t_v, Fy_t = train_test_split(F_x, F_y, random_state=seed, test_size=test_size)

In [ ]:
t_list = []
for i in range(t_d.shape[0]):
  t_list.append(t_d[i,v_p[i,0]:])

In [ ]:
tx, ty = build_sequences2(t_list, window=200, telescope=18, step=200)
print(tx.shape)

(23016, 200)


# Plan 1

In this file we tested a normal CNN network, just six Conv1D layers and one dense layer

First we trained the network with all categories simultaneously and then with each category individually to specialise the network for each category

In [ ]:
def simple_model():
    input_layer = tfkl.Input(shape=(200,1), name='input_layer')

    x = tfkl.Conv1D(32, 10, padding='same', activation='relu', name='conv1')(input_layer)
    x = tfkl.Conv1D(32, 10, padding='same', activation='relu', name='conv2')(x)
    x = tfkl.AveragePooling1D(3, name='avg1')(x)
    x = tfkl.Conv1D(64, 10, padding='same', activation='relu', name='conv3')(x)
    x = tfkl.Conv1D(64, 10, padding='same', activation='relu', name='conv4')(x)
    x= tfkl.AveragePooling1D(3, name='avg2')(x)
    x = tfkl.Conv1D(128, 10, padding='same', activation='relu', name='conv5')(x)
    x = tfkl.Conv1D(128, 10, padding='same', activation='relu', name='conv6')(x)

    gap = tfkl.GlobalAveragePooling1D(name='gap')(x)

    output_layer = tfkl.Dense(18, activation='linear', name='output')(gap)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='simp_model')
    return model

## All Categories

In [ ]:
model = simple_model()
model.summary()

Model: "simp_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 200, 1)]          0         
                                                                 
 conv1 (Conv1D)              (None, 200, 32)           352       
                                                                 
 conv2 (Conv1D)              (None, 200, 32)           10272     
                                                                 
 avg1 (AveragePooling1D)     (None, 66, 32)            0         
                                                                 
 conv3 (Conv1D)              (None, 66, 64)            20544     
                                                                 
 conv4 (Conv1D)              (None, 66, 64)            41024     
                                                                 
 avg2 (AveragePooling1D)     (None, 22, 64)            0

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
model.compile(loss='mae', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
save_model(model, 'model1', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

In [ ]:
model = load_model('Cate/model1')[0]

In [ ]:
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 256,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history

In [ ]:
save_model(model, 'model2', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

## Connecting with an attention-layer to get the Attention-model from the lessons

In [ ]:
model = load_model('Cate/model1')[0]

In [ ]:
model.compile('adam', loss='mse', metrics=['mse', 'mae'])

In [ ]:
def build_model(model):
  input_size = (200,1)
  output_size = 18

  dense_dim = 200
  embed_dim = 18
  latent_dim = 100
  num_heads = 8

  coder_input_size = (18,1)

  input = tfk.Input(shape=input_size, name='INPUT')
  x = tf.expand_dims(model(input), axis=2)

  dense_proj = tfk.Sequential([tfkl.Dense(dense_dim, activation='relu'), tfkl.Dense(embed_dim),])

  attention_output = tfkl.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(query=x, value=x, key=x)
  proj_input = tfkl.LayerNormalization(name='Norm1')(x + attention_output)
  proj_output = dense_proj(proj_input)
  enc = tfkl.LayerNormalization(name='Norm2')(proj_input + proj_output)


  attention_output = tfkl.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(query=x,value=enc,key=enc)
  out_2 = x + attention_output


  dense_out = tfk.Sequential(
            [
                tfkl.GlobalAveragePooling1D(name='gap'),
                tfkl.Dense(100, activation='relu'),
                tfkl.Dense(18),
            ])(out_2)

  return tfk.Model(input, dense_out, name='transformer')

In [ ]:
model.trainable = False

In [ ]:
trans = build_model(model)

In [ ]:
trans.compile('adam', loss='mse', metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = tx,
    y = ty,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.1, min_lr=1e-5)
    ]
).history

Epoch 1/200
162/162 [==============================] - 52s 312ms/step - loss: 0.0055 - mse: 0.0055 - mae: 0.0499 - val_loss: 0.0115 - val_mse: 0.0115 - val_mae: 0.0673 - lr: 0.0010
Epoch 2/200
162/162 [==============================] - 53s 328ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0472 - val_loss: 0.0118 - val_mse: 0.0118 - val_mae: 0.0698 - lr: 0.0010
Epoch 3/200
162/162 [==============================] - 48s 294ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0476 - val_loss: 0.0119 - val_mse: 0.0119 - val_mae: 0.0697 - lr: 0.0010
Epoch 4/200
162/162 [==============================] - 47s 291ms/step - loss: 0.0045 - mse: 0.0045 - mae: 0.0457 - val_loss: 0.0119 - val_mse: 0.0119 - val_mae: 0.0703 - lr: 0.0010
Epoch 5/200
162/162 [==============================] - 47s 291ms/step - loss: 0.0045 - mse: 0.0045 - mae: 0.0457 - val_loss: 0.0120 - val_mse: 0.0120 - val_mae: 0.0696 - lr: 0.0010
Epoch 6/200
162/162 [==============================] - 46s 285ms/step - loss: 0.0043 - mse: 0.0

it seems to overfit

## Cate F

In [ ]:
model = load_model('Cate/model2')[0]

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
F_x, F_y = build_sequences(t_d[F], v_p[F], window=200, telescope=18, step = 10)
print(f'Number of Training: {F_x.shape[0]}')
test_size = 0.2
Fx_t_v, Fx_t, Fy_t_v, Fy_t = train_test_split(F_x, F_y, random_state=seed, test_size=test_size)

Number of Training: 1245


In [ ]:
history = model.fit(
    x = Fx_t_v,
    y = Fy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ]
).history

Epoch 1/200
14/14 [==============================] - 4s 159ms/step - loss: 0.0079 - mse: 0.0079 - mae: 0.0517 - val_loss: 0.0092 - val_mse: 0.0092 - val_mae: 0.0456 - lr: 0.0010
Epoch 2/200
14/14 [==============================] - 2s 143ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0365 - val_loss: 0.0081 - val_mse: 0.0081 - val_mae: 0.0408 - lr: 0.0010
Epoch 3/200
14/14 [==============================] - 2s 134ms/step - loss: 0.0043 - mse: 0.0043 - mae: 0.0350 - val_loss: 0.0076 - val_mse: 0.0076 - val_mae: 0.0381 - lr: 0.0010
Epoch 4/200
14/14 [==============================] - 2s 136ms/step - loss: 0.0043 - mse: 0.0043 - mae: 0.0358 - val_loss: 0.0074 - val_mse: 0.0074 - val_mae: 0.0378 - lr: 0.0010
Epoch 5/200
14/14 [==============================] - 3s 219ms/step - loss: 0.0038 - mse: 0.0038 - mae: 0.0305 - val_loss: 0.0076 - val_mse: 0.0076 - val_mae: 0.0397 - lr: 0.0010
Epoch 6/200
14/14 [==============================] - 3s 227ms/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0306 

In [ ]:
save_model(model, 'model1_F', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

In [ ]:
model = load_model('Cate/model1_F')[0]

In [ ]:
model.compile(tfk.optimizers.Adam(learning_rate=0.001)
    , loss='mse', metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = Fx_t_v,
    y = Fy_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.19, min_lr=1e-5)
    ]
).history

Epoch 1/200
7/7 [==============================] - 5s 463ms/step - loss: 0.0026 - mse: 0.0026 - mae: 0.0280 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0353 - lr: 0.0010
Epoch 2/200
7/7 [==============================] - 2s 250ms/step - loss: 0.0022 - mse: 0.0022 - mae: 0.0237 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0361 - lr: 0.0010
Epoch 3/200
7/7 [==============================] - 2s 253ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0196 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0343 - lr: 0.0010
Epoch 4/200
7/7 [==============================] - 2s 258ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0179 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0327 - lr: 0.0010
Epoch 5/200
7/7 [==============================] - 2s 246ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0167 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae: 0.0345 - lr: 0.0010
Epoch 6/200
7/7 [==============================] - 2s 249ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0181 - val_loss: 

In [ ]:
save_model(model, 'model2_F', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

In [ ]:
model = load_model('Cate/model2_F')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])

F_x, F_y = build_sequences(t_d[F], v_p[F], window=200, telescope=18, step = 3)
print(f'Number of Training: {F_x.shape[0]}')
test_size = 0.2
Fx_t_v, Fx_t, Fy_t_v, Fy_t = train_test_split(F_x, F_y, random_state=seed, test_size=test_size)

history = model.fit(
    x = Fx_t_v,
    y = Fy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model3_F', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Number of Training: 4055
Epoch 1/200
46/46 [==============================] - 8s 145ms/step - loss: 0.0033 - mse: 0.0033 - mae: 0.0296 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0245 - lr: 0.0010
Epoch 2/200
46/46 [==============================] - 8s 176ms/step - loss: 0.0024 - mse: 0.0024 - mae: 0.0240 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0249 - lr: 0.0010
Epoch 3/200
46/46 [==============================] - 6s 133ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0238 - val_loss: 0.0025 - val_mse: 0.0025 - val_mae: 0.0275 - lr: 0.0010
Epoch 4/200
46/46 [==============================] - 9s 207ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0251 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0251 - lr: 0.0010
Epoch 5/200
46/46 [==============================] - 6s 132ms/step - loss: 0.0024 - mse: 0.0024 - mae: 0.0262 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0256 - lr: 0.0010
Epoch 6/200
46/46 [==============================] - 8s 177ms/step - loss: 0.0025 - m

## Cate A

In [ ]:
model = load_model('Cate/model2')[0]

In [ ]:
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [ ]:
A_x, A_y = build_sequences(t_d[A], v_p[A], window=200, telescope=18, step = 190)
print(f'Number of Training: {A_x.shape[0]}')
test_size = 0.2
Ax_t_v, Ax_t, Ay_t_v, Ay_t = train_test_split(A_x, A_y, random_state=seed, test_size=test_size)

Number of Training: 4934


In [ ]:
history = model.fit(
    x = Ax_t_v,
    y = Ay_t_v,
    batch_size = 256,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history

Epoch 1/200
14/14 [==============================] - 11s 700ms/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0649 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0575 - lr: 0.0010
Epoch 2/200
14/14 [==============================] - 8s 552ms/step - loss: 0.0051 - mse: 0.0051 - mae: 0.0500 - val_loss: 0.0052 - val_mse: 0.0052 - val_mae: 0.0497 - lr: 0.0010
Epoch 3/200
14/14 [==============================] - 11s 789ms/step - loss: 0.0045 - mse: 0.0045 - mae: 0.0466 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0487 - lr: 0.0010
Epoch 4/200
14/14 [==============================] - 7s 536ms/step - loss: 0.0042 - mse: 0.0042 - mae: 0.0444 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0483 - lr: 0.0010
Epoch 5/200
14/14 [==============================] - 9s 652ms/step - loss: 0.0039 - mse: 0.0039 - mae: 0.0433 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0482 - lr: 0.0010
Epoch 6/200
14/14 [==============================] - 13s 970ms/step - loss: 0.0037 - mse: 0.0037 - mae: 0.04

In [ ]:
save_model(model, 'model1_A', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

In [ ]:
model = load_model('Cate/model1_A')[0]

In [ ]:
model.compile(optimizer=tfk.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mse', 'mae'])

In [ ]:
history = model.fit(
    x = Ax_t_v,
    y = Ay_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history

Epoch 1/200
28/28 [==============================] - 10s 294ms/step - loss: 0.0044 - mse: 0.0044 - mae: 0.0483 - val_loss: 0.0052 - val_mse: 0.0052 - val_mae: 0.0500 - lr: 0.0010
Epoch 2/200
28/28 [==============================] - 10s 338ms/step - loss: 0.0029 - mse: 0.0029 - mae: 0.0378 - val_loss: 0.0049 - val_mse: 0.0049 - val_mae: 0.0483 - lr: 0.0010
Epoch 3/200
28/28 [==============================] - 9s 308ms/step - loss: 0.0026 - mse: 0.0026 - mae: 0.0359 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0486 - lr: 0.0010
Epoch 4/200
28/28 [==============================] - 12s 411ms/step - loss: 0.0025 - mse: 0.0025 - mae: 0.0352 - val_loss: 0.0049 - val_mse: 0.0049 - val_mae: 0.0481 - lr: 0.0010
Epoch 5/200
28/28 [==============================] - 10s 343ms/step - loss: 0.0024 - mse: 0.0024 - mae: 0.0349 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0485 - lr: 0.0010
Epoch 6/200
28/28 [==============================] - 7s 243ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0

In [ ]:
save_model(model, 'model2_A', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

In [ ]:
model.compile(tfk.optimizers.AdamW(learning_rate=0.001), loss='mse', metrics=['mse', 'mae'])

In [ ]:
A_x, A_y = build_sequences(t_d[A], v_p[A], window=200, telescope=18, step = 150)
print(f'Number of Training: {A_x.shape[0]}')
test_size = 0.2
Ax_t_v, Ax_t, Ay_t_v, Ay_t = train_test_split(A_x, A_y, random_state=seed, test_size=test_size)

Number of Training: 5062


In [ ]:
history = model.fit(
    x = Ax_t_v,
    y = Ay_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history

Epoch 1/200
29/29 [==============================] - 12s 348ms/step - loss: 0.0064 - mse: 0.0064 - mae: 0.0582 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0472 - lr: 0.0010
Epoch 2/200
29/29 [==============================] - 7s 246ms/step - loss: 0.0039 - mse: 0.0039 - mae: 0.0435 - val_loss: 0.0041 - val_mse: 0.0041 - val_mae: 0.0427 - lr: 0.0010
Epoch 3/200
29/29 [==============================] - 10s 340ms/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0412 - val_loss: 0.0041 - val_mse: 0.0041 - val_mae: 0.0426 - lr: 0.0010
Epoch 4/200
29/29 [==============================] - 7s 245ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0394 - val_loss: 0.0040 - val_mse: 0.0040 - val_mae: 0.0416 - lr: 0.0010
Epoch 5/200
29/29 [==============================] - 10s 334ms/step - loss: 0.0030 - mse: 0.0030 - mae: 0.0383 - val_loss: 0.0039 - val_mse: 0.0039 - val_mae: 0.0420 - lr: 0.0010
Epoch 6/200
29/29 [==============================] - 7s 248ms/step - loss: 0.0028 - mse: 0.0028 - mae: 0.03

In [ ]:
save_model(model, 'model3_A', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

## Cate B

In [ ]:
B_x, B_y = build_sequences(t_d[B], v_p[B], window=200, telescope=18, step = 80)
print(f'Number of Training: {B_x.shape[0]}')
test_size = 0.2
Bx_t_v, Bx_t, By_t_v, By_t = train_test_split(B_x, B_y, random_state=seed, test_size=test_size)

Number of Training: 4944


In [ ]:
model = load_model('Cate/model2')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

history = model.fit(
    x = Bx_t_v,
    y = By_t_v,
    batch_size = 256,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model1_B', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model

Epoch 1/200
14/14 [==============================] - 14s 898ms/step - loss: 0.0121 - mse: 0.0121 - mae: 0.0716 - val_loss: 0.0100 - val_mse: 0.0100 - val_mae: 0.0660 - lr: 0.0010
Epoch 2/200
14/14 [==============================] - 9s 638ms/step - loss: 0.0104 - mse: 0.0104 - mae: 0.0659 - val_loss: 0.0103 - val_mse: 0.0103 - val_mae: 0.0679 - lr: 0.0010
Epoch 3/200
14/14 [==============================] - 10s 661ms/step - loss: 0.0097 - mse: 0.0097 - mae: 0.0632 - val_loss: 0.0097 - val_mse: 0.0097 - val_mae: 0.0637 - lr: 0.0010
Epoch 4/200
14/14 [==============================] - 9s 660ms/step - loss: 0.0093 - mse: 0.0093 - mae: 0.0617 - val_loss: 0.0095 - val_mse: 0.0095 - val_mae: 0.0639 - lr: 0.0010
Epoch 5/200
14/14 [==============================] - 8s 546ms/step - loss: 0.0088 - mse: 0.0088 - mae: 0.0608 - val_loss: 0.0096 - val_mse: 0.0096 - val_mae: 0.0628 - lr: 0.0010
Epoch 6/200
14/14 [==============================] - 8s 571ms/step - loss: 0.0083 - mse: 0.0083 - mae: 0.058

In [ ]:
model = load_model('Cate/model1_B')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

history = model.fit(
    x = Bx_t_v,
    y = By_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model2_B', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model

Epoch 1/200
28/28 [==============================] - 9s 258ms/step - loss: 0.0079 - mse: 0.0079 - mae: 0.0593 - val_loss: 0.0098 - val_mse: 0.0098 - val_mae: 0.0634 - lr: 0.0010
Epoch 2/200
28/28 [==============================] - 14s 505ms/step - loss: 0.0062 - mse: 0.0062 - mae: 0.0508 - val_loss: 0.0090 - val_mse: 0.0090 - val_mae: 0.0607 - lr: 0.0010
Epoch 3/200
28/28 [==============================] - 16s 570ms/step - loss: 0.0058 - mse: 0.0058 - mae: 0.0491 - val_loss: 0.0095 - val_mse: 0.0095 - val_mae: 0.0622 - lr: 0.0010
Epoch 4/200
28/28 [==============================] - 20s 717ms/step - loss: 0.0058 - mse: 0.0058 - mae: 0.0492 - val_loss: 0.0097 - val_mse: 0.0097 - val_mae: 0.0634 - lr: 0.0010
Epoch 5/200
28/28 [==============================] - 15s 555ms/step - loss: 0.0056 - mse: 0.0056 - mae: 0.0493 - val_loss: 0.0097 - val_mse: 0.0097 - val_mae: 0.0619 - lr: 0.0010
Epoch 6/200
28/28 [==============================] - 8s 271ms/step - loss: 0.0053 - mse: 0.0053 - mae: 0.0

In [ ]:
model = load_model('Cate/model2_B')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])

B_x, B_y = build_sequences(t_d[B], v_p[B], window=200, telescope=18, step = 70)
print(f'Number of Training: {B_x.shape[0]}')
test_size = 0.2
Bx_t_v, Bx_t, By_t_v, By_t = train_test_split(B_x, B_y, random_state=seed, test_size=test_size)

history = model.fit(
    x = Bx_t_v,
    y = By_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model3_B', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Number of Training: 5201
Epoch 1/200
30/30 [==============================] - 12s 355ms/step - loss: 0.0101 - mse: 0.0101 - mae: 0.0666 - val_loss: 0.0078 - val_mse: 0.0078 - val_mae: 0.0588 - lr: 0.0010
Epoch 2/200
30/30 [==============================] - 7s 243ms/step - loss: 0.0080 - mse: 0.0080 - mae: 0.0582 - val_loss: 0.0070 - val_mse: 0.0070 - val_mae: 0.0539 - lr: 0.0010
Epoch 3/200
30/30 [==============================] - 13s 452ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0549 - val_loss: 0.0076 - val_mse: 0.0076 - val_mae: 0.0578 - lr: 0.0010
Epoch 4/200
30/30 [==============================] - 15s 498ms/step - loss: 0.0067 - mse: 0.0067 - mae: 0.0536 - val_loss: 0.0081 - val_mse: 0.0081 - val_mae: 0.0626 - lr: 0.0010
Epoch 5/200
30/30 [==============================] - 11s 372ms/step - loss: 0.0064 - mse: 0.0064 - mae: 0.0535 - val_loss: 0.0069 - val_mse: 0.0069 - val_mae: 0.0556 - lr: 0.0010
Epoch 6/200
30/30 [==============================] - 9s 283ms/step - loss: 0.0058

## Cate C

In [ ]:
C_x, C_y = build_sequences(t_d[C], v_p[C], window=200, telescope=18, step = 150)
print(f'Number of Training: {C_x.shape[0]}')
test_size = 0.2
Cx_t_v, Cx_t, Cy_t_v, Cy_t = train_test_split(C_x, C_y, random_state=seed, test_size=test_size)
model = load_model('Cate/model2')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

history = model.fit(
    x = Cx_t_v,
    y = Cy_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model1_C', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model
model = load_model('Cate/model1_C')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
history = model.fit(
    x = Cx_t_v,
    y = Cy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model2_C', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model

model = load_model('Cate/model2_C')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])

C_x, C_y = build_sequences(t_d[C], v_p[C], window=200, telescope=18, step = 100)
print(f'Number of Training: {C_x.shape[0]}')
test_size = 0.2
Cx_t_v, Cx_t, Cy_t_v, Cy_t = train_test_split(C_x, C_y, random_state=seed, test_size=test_size)

history = model.fit(
    x = Cx_t_v,
    y = Cy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model3_C', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Number of Training: 5201
Epoch 1/200
15/15 [==============================] - 9s 517ms/step - loss: 0.0080 - mse: 0.0080 - mae: 0.0614 - val_loss: 0.0069 - val_mse: 0.0069 - val_mae: 0.0562 - lr: 0.0010
Epoch 2/200
15/15 [==============================] - 10s 629ms/step - loss: 0.0065 - mse: 0.0065 - mae: 0.0545 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0537 - lr: 0.0010
Epoch 3/200
15/15 [==============================] - 7s 468ms/step - loss: 0.0058 - mse: 0.0058 - mae: 0.0507 - val_loss: 0.0061 - val_mse: 0.0061 - val_mae: 0.0518 - lr: 0.0010
Epoch 4/200
15/15 [==============================] - 10s 647ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0490 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0509 - lr: 0.0010
Epoch 5/200
15/15 [==============================] - 7s 479ms/step - loss: 0.0051 - mse: 0.0051 - mae: 0.0478 - val_loss: 0.0060 - val_mse: 0.0060 - val_mae: 0.0516 - lr: 0.0010
Epoch 6/200
15/15 [==============================] - 10s 639ms/step - loss: 0.0049 

## Cate D

In [ ]:
D_x, D_y = build_sequences(t_d[D], v_p[D], window=200, telescope=18, step = 150)
print(f'Number of Training: {D_x.shape[0]}')
test_size = 0.2
Dx_t_v, Dx_t, Dy_t_v, Dy_t = train_test_split(D_x, D_y, random_state=seed, test_size=test_size)
model = load_model('Cate/model2')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

history = model.fit(
    x = Dx_t_v,
    y = Dy_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model1_D', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model
model = load_model('Cate/model1_D')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
history = model.fit(
    x = Dx_t_v,
    y = Dy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model2_D', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model

model = load_model('Cate/model2_D')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])

D_x, D_y = build_sequences(t_d[D], v_p[D], window=200, telescope=18, step = 100)
print(f'Number of Training: {D_x.shape[0]}')
test_size = 0.2
Dx_t_v, Dx_t, Dy_t_v, Dy_t = train_test_split(D_x, D_y, random_state=seed, test_size=test_size)

history = model.fit(
    x = Dx_t_v,
    y = Dy_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model3_D', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Number of Training: 6777
Epoch 1/200
39/39 [==============================] - 18s 395ms/step - loss: 0.0065 - mse: 0.0065 - mae: 0.0551 - val_loss: 0.0055 - val_mse: 0.0055 - val_mae: 0.0499 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 11s 282ms/step - loss: 0.0056 - mse: 0.0056 - mae: 0.0498 - val_loss: 0.0057 - val_mse: 0.0057 - val_mae: 0.0519 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 19s 496ms/step - loss: 0.0053 - mse: 0.0053 - mae: 0.0487 - val_loss: 0.0055 - val_mse: 0.0055 - val_mae: 0.0509 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 22s 555ms/step - loss: 0.0051 - mse: 0.0051 - mae: 0.0483 - val_loss: 0.0055 - val_mse: 0.0055 - val_mae: 0.0504 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 21s 526ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0475 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0584 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 19s 496ms/step - loss: 0.00

## Cate E

In [ ]:
E_x, E_y = build_sequences(t_d[E], v_p[E], window=200, telescope=18, step = 150)
print(f'Number of Training: {E_x.shape[0]}')
test_size = 0.2
Ex_t_v, Ex_t, Ey_t_v, Ey_t = train_test_split(E_x, E_y, random_state=seed, test_size=test_size)
model = load_model('Cate/model2')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

history = model.fit(
    x = Ex_t_v,
    y = Ey_t_v,
    batch_size = 128,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model1_E', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model
model = load_model('Cate/model1_E')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
history = model.fit(
    x = Ex_t_v,
    y = Ey_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model2_E', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)
del model

model = load_model('Cate/model2_E')[0]
model.compile(loss='mse', optimizer=tfk.optimizers.AdamW(learning_rate=0.001), metrics=['mse', 'mae'])

E_x, E_y = build_sequences(t_d[E], v_p[E], window=200, telescope=18, step = 100)
print(f'Number of Training: {E_x.shape[0]}')
test_size = 0.2
Ex_t_v, Ex_t, Ey_t_v, Ey_t = train_test_split(E_x, E_y, random_state=seed, test_size=test_size)

history = model.fit(
    x = Ex_t_v,
    y = Ey_t_v,
    batch_size = 64,
    epochs = 200,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=7, factor=0.19, min_lr=1e-5)
    ]
).history
save_model(model, 'model3_E', 'Cate', history['mse'], history['val_mse'], history['mae'], history['val_mae'], plot=False)

Number of Training: 4630
Epoch 1/200
27/27 [==============================] - 10s 332ms/step - loss: 0.0047 - mse: 0.0047 - mae: 0.0468 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0444 - lr: 0.0010
Epoch 2/200
27/27 [==============================] - 8s 310ms/step - loss: 0.0039 - mse: 0.0039 - mae: 0.0422 - val_loss: 0.0044 - val_mse: 0.0044 - val_mae: 0.0442 - lr: 0.0010
Epoch 3/200
27/27 [==============================] - 11s 403ms/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0411 - val_loss: 0.0049 - val_mse: 0.0049 - val_mae: 0.0479 - lr: 0.0010
Epoch 4/200
27/27 [==============================] - 7s 245ms/step - loss: 0.0034 - mse: 0.0034 - mae: 0.0400 - val_loss: 0.0048 - val_mse: 0.0048 - val_mae: 0.0466 - lr: 0.0010
Epoch 5/200
27/27 [==============================] - 9s 319ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0391 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0472 - lr: 0.0010
Epoch 6/200
27/27 [==============================] - 7s 265ms/step - loss: 0.0030 -

## Validation

In [ ]:
model = load_model('Cate/model2')[0]
print(f'Cate A:{model.evaluate(Ax_t, Ay_t, verbose=0)}')
print(f'Cate B:{model.evaluate(Bx_t, By_t, verbose=0)}')
print(f'Cate C:{model.evaluate(Cx_t, Cy_t, verbose=0)}')
print(f'Cate D:{model.evaluate(Dx_t, Dy_t, verbose=0)}')
print(f'Cate E:{model.evaluate(Ex_t, Ey_t, verbose=0)}')
print(f'Cate F:{model.evaluate(Fx_t, Fy_t, verbose=0)}')

modelA = load_model('Cate/model3_A')[0]
print(f'Cate A, modelA:{modelA.evaluate(Ax_t, Ay_t, verbose=0)}')
modelB = load_model('Cate/model3_B')[0]
print(f'Cate B, modelB:{modelB.evaluate(Bx_t, By_t, verbose=0)}')
modelC = load_model('Cate/model3_C')[0]
print(f'Cate C, modelC:{modelC.evaluate(Cx_t, Cy_t, verbose=0)}')
modelD = load_model('Cate/model3_D')[0]
print(f'Cate D, modelD:{modelD.evaluate(Dx_t, Dy_t, verbose=0)}')
modelE = load_model('Cate/model3_E')[0]
print(f'Cate E, modelE:{modelE.evaluate(Ex_t, Ey_t, verbose=0)}')
modelF = load_model('Cate/model3_F')[0]
print(f'Cate F, modelF:{modelF.evaluate(Fx_t, Fy_t, verbose=0)}')

Cate A:[0.005395933985710144, 0.005395933985710144, 0.05062131956219673]
Cate B:[0.01083130668848753, 0.01083130668848753, 0.06436609476804733]
Cate C:[0.007284738589078188, 0.007284738589078188, 0.05596916750073433]
Cate D:[0.007261027116328478, 0.007261027116328478, 0.05589313805103302]
Cate E:[0.006190949585288763, 0.006190949585288763, 0.05012274533510208]
Cate F:[0.010768625885248184, 0.010768625885248184, 0.057763420045375824]
Cate A, modelA:[0.004355693701654673, 0.004355693701654673, 0.04454798251390457]
Cate B, modelB:[0.00827246904373169, 0.00827246904373169, 0.055740196257829666]
Cate C, modelC:[0.006346042733639479, 0.006346042733639479, 0.05047645419836044]
Cate D, modelD:[0.006143788807094097, 0.006143788807094097, 0.0493631549179554]
Cate E, modelE:[0.005029665306210518, 0.005029665306210518, 0.04153698682785034]
Cate F, modelF:[0.0015558165032416582, 0.0015558165032416582, 0.015301434323191643]


We got in each category with the individualised network better results than with the general Network